In [10]:
import cv2
import mediapipe as mp
import pyautogui
from pynput.mouse import Button,Controller
import random
import os
import time

In [11]:
screen_width,screen_height = pyautogui.size()
mouse = Controller()
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode = False,
    model_complexity = 1,
    min_detection_confidence = 0.7,
    min_tracking_confidence = 0.7,
    max_num_hands = 1
)

In [12]:
import numpy as np

def get_angle(a,b,c):
    radians = np.arctan2(c[1] - b[1],c[0] - b[0]) - np.arctan2(a[1] - b[1],a[0] - b[0])
    angle = np.abs(np.degrees(radians))
    return angle

def get_distance(landmarks_list):
    if len(landmarks_list) < 2:  # Ensure at least two landmarks are present
        return None
    (x1, y1), (x2, y2) = landmarks_list[0], landmarks_list[1]
    L = np.hypot(x2 - x1, y2 - y1)
    return np.interp(L, [0, 1], [0, 1000])  # Ensure L is properly mapped

In [13]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]

    return None

In [14]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y * screen_height)
        pyautogui.moveTo(x,y)

In [15]:
def is_left_click(landmarks_list, thumb_index_dist):
    angle1 = get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8])
    # print(f"Left Click Angle: {angle1}, Thumb-Index Distance: {thumb_index_dist}")
    return angle1 < 50 and thumb_index_dist > 50 

def is_right_click(landmarks_list, thumb_index_dist):
    angle1 = get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12])  # Middle finger bend
    angle2 = get_angle(landmarks_list[13], landmarks_list[14], landmarks_list[16])  # Ring finger bend
    # print(f"Right Click Angles: {angle1}, {angle2}, Thumb-Index Distance: {thumb_index_dist}")
    return angle1 < 50 and angle2 < 90 and thumb_index_dist > 50  # Middle finger bent & thumb away

def is_double_click(landmarks_list, thumb_index_dist):
    angle1 = get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8])  # Index finger bend
    angle2 = get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12])  # Middle finger bend
    # print(f"Double Click Angles: {angle1}, {angle2}, Thumb-Index Distance: {thumb_index_dist}")
    return angle1 < 50 and angle2 < 50 and thumb_index_dist > 50  # Both fingers bent & thumb away

def is_screenshort(landmarks_list,thumb_index_dist):
    # Index finger bend,Middle finger bend and thumb are close to the index finger
    angle1 = get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8])
    angle2 = get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12])
    # print(f"Screenshot Angles: {angle1}, {angle2}")
    return (angle1 < 50 and angle2 < 50 and thumb_index_dist < 50)

dragging = False  

def is_pinch(landmarks_list, threshold=0.05):
    """Detect if index finger and thumb are pinched (close together)."""
    thumb_x, thumb_y = landmarks_list[4]  # Thumb tip
    index_x, index_y = landmarks_list[8]  # Index finger tip

    distance = ((thumb_x - index_x) ** 2 + (thumb_y - index_y) ** 2) ** 0.5
    return distance < threshold  # If distance is small, fingers are pinched

def drag_and_drop(landmarks_list):
    """Handles drag & drop functionality using hand gestures."""
    global dragging

    # Get index finger tip coordinates
    index_x, index_y = landmarks_list[8]  # Assuming normalized values (0-1)

    # Convert to screen coordinates
    cursor_x = int(index_x * screen_width)
    cursor_y = int(index_y * screen_height)

    if is_pinch(landmarks_list):  # If pinch is detected (start dragging)
        if not dragging:
            print("Moving to target...")
            pyautogui.moveTo(cursor_x, cursor_y, duration=0.2)  # Move cursor to target
            time.sleep(0.1)  # Short delay before dragging

            print("Drag started at:", cursor_x, cursor_y)
            pyautogui.mouseDown()  # Press mouse button to grab the item
            dragging = True
        else:
            print("Dragging to:", cursor_x, cursor_y)
            pyautogui.moveTo(cursor_x, cursor_y, duration=0.1)  # Smooth dragging

    elif dragging:  # If pinch is released (drop the item)
        print("Drop at:", cursor_x, cursor_y)
        pyautogui.mouseUp()  # Release the mouse
        dragging = False  # Reset dragging state
        time.sleep(0.2)  # Small delay to prevent false triggers

In [16]:
def detect_gestures(frame,landmarks_list,processed):
    global dragging
    save_directory = r"E:\Virtual_Mouse_Sreenshorts"
    if len(landmarks_list) >= 21:
        index_finger_tip = find_finger_tip(processed)
        thumb_index_dist = get_distance([landmarks_list[4],landmarks_list[5]])

        if get_distance([landmarks_list[4], landmarks_list[5]]) < 50 and get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) > 90:
            move_mouse(index_finger_tip)

        # Left Click
        elif is_left_click(landmarks_list,thumb_index_dist):
            # print("Left Click detected")
            mouse.press(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame,"Left Click",(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

        # Right Click
        elif is_right_click(landmarks_list,thumb_index_dist):
            # print("Right Click detected")
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame,"Right Click",(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)

        # Double Click
        elif is_double_click(landmarks_list,thumb_index_dist):
            # print("Double Click detected")
            pyautogui.doubleClick()
            cv2.putText(frame,"Double Click",(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),2)
            
        # Screen_Shorts
        elif is_screenshort(landmarks_list,thumb_index_dist):
            # print("Screenshot detected")
            im1 = pyautogui.screenshot()
            label = random.randint(1,1000)
            file_path = os.path.join(save_directory, f"my_screenshot_{label}.png")
            im1.save(file_path)
            cv2.putText(frame,"Screenshort Taken",(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),2)

        elif is_pinch(landmarks_list):  # Drag & Drop functionality
            drag_and_drop(landmarks_list)  # Call the drag function
    
            # Display "Dragging..." while moving
            cv2.putText(frame, "Dragging...", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        elif dragging:              # Ensure drop happens when pinch i     
            print("Drop detected!") 
            pyautogui.mouseUp()     # Release the mouse
            dragging = False

In [17]:
def main():
    cap = cv2.VideoCapture(0)
    draw = mp.solutions.drawing_utils
    try:
        while cap.isOpened():
            r,frame = cap.read()

            if not r:
                break

            frame = cv2.flip(frame,1)                            # Merraring the Frame
            frameRGB = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)     # Converting Color BGR To RGB For Mediapipe
            processed = hands.process(frameRGB)                  # It processes the Frame and Detect All the Land Marks

            # Recive All the Landmarks
            landmarks_list = []
            
            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]
                draw.draw_landmarks(frame,hand_landmarks, mpHands.HAND_CONNECTIONS)    # It is used to see the Landmarks

                for lm in hand_landmarks.landmark:
                    landmarks_list.append((lm.x,lm.y))

                # print(landmarks_list)

            detect_gestures(frame,landmarks_list,processed)
            
            cv2.imshow('Frame',frame)
        
            if cv2.waitKey(1) & 0xFF == ord('q'):                # Closing Logic
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

Moving to target...
Drag started at: 1043 665
Dragging to: 1050 657
Dragging to: 1063 661
Drop detected!
Moving to target...
Drag started at: 1085 614
Drop detected!
Moving to target...
Drag started at: 1085 626
Dragging to: 1075 600
Dragging to: 1127 600
Dragging to: 1177 591
Drop detected!
Moving to target...
Drag started at: 1292 593
Drop detected!
Moving to target...
Drag started at: 1087 648
Drop detected!
Moving to target...
Drag started at: 1072 647
Drop detected!
Moving to target...
Drag started at: 1100 591
Drop detected!
Moving to target...
Drag started at: 1092 599
Drop detected!
Moving to target...
Drag started at: 1118 544
Drop detected!
Moving to target...
Drag started at: 1132 526
Drop detected!
Moving to target...
Drag started at: 1108 542
Dragging to: 1196 535
Dragging to: 1179 537
Drop detected!
Moving to target...
Drag started at: 1013 784
Drop detected!
